In [ ]:
import numpy as np
import scipy as sp
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Import data

In [ ]:
bids = pd.read_csv('data/bids.csv', sep=',')
bids.head()

In [297]:
bids = pd.read_csv('data/bids_normalized.csv', sep=',')
bids.head()

,bid_id,bidder_id,auction,merchandise,device,time,country,ip,url,time_normalized,outcome
0,0,8dac2b259fd1c6d1120e519fb1ac14fbqvax8,ewmzr,jewelry,phone0,9759243157894736,us,69.166.231.58,vasstdc27m7nks3,2419200.0,NaN
1,1,668d393e858e8126275433046bbd35c6tywop,aeqok,furniture,phone1,9759243157894736,in,50.201.125.84,jmqlhflrzwuay9c,2419200.0,0.0
2,2,aa5f360084278b35d746fa6af3a7a1a5ra3xe,wa00e,home goods,phone2,9759243157894736,py,112.54.208.157,vasstdc27m7nks3,2419200.0,NaN
3,3,3939ac3ef7d472a59a9c5f893dd3e39fh9ofi,jefix,jewelry,phone4,9759243157894736,in,18.99.175.133,vasstdc27m7nks3,2419200.0,NaN
4,4,8393c48eaf4b8fa96886edc7cf27b372dsibi,jefix,jewelry,phone5,9759243157894736,in,145.138.5.37,vasstdc27m7nks3,2419200.0,NaN


In [ ]:
b_train = pd.read_csv('data/train.csv', sep=',')
b_train.head()

In [310]:
b_train = pd.read_csv('data/b_train.csv', sep=',')
b_train.head(10)

,bidder_id,outcome,mean_time_diff,total_bids_count,total_auctions_count,mean_time_diff2
0,91a3c57b13234af24875c56fb7e2b2f4rb56a,0.0,10851.13,24,18,10673.375000
1,624f258b49e77713fc34034560f93fb3hu3jo,0.0,61438.00,3,1,2.666667
2,1c5f4fc669099bfbfac515cd26997bd12ruaj,0.0,45201.00,4,4,219.000000
3,4bee9aba2abda51bf43d639013d6efe12iycd,0.0,NaN,1,1,330.000000
4,4ab12bc61c82ddd9c2d65e60555808acqgos1,0.0,1477.80,155,23,35.206452
5,7eaefc97fbf6af12e930528151f86eb91bafh,0.0,NaN,1,1,0.000000
6,25558d24bca82beef0f9db4ba1fe2045ynnvq,0.0,30379.14,8,8,181.750000
7,88ae7a35e374a6fddd079ebb28c822eeohwse,0.0,3222.55,34,10,148.794118
8,57db69e32163f3e486dc6ef7d615aa12usje6,0.0,1296178.00,2,2,9.000000
9,d1be739798ba0745a1fd72ac918a9f1929hei,0.0,100964.38,14,10,127.428571


In [ ]:
b_test = pd.read_csv('data/test.csv', sep=',')
b_test.head()

# PREPROCESSING

# 1) bids_normalized table

In [308]:
bids.head()

,bid_id,bidder_id,auction,time,time_normalized,outcome
0,0,8dac2b259fd1c6d1120e519fb1ac14fbqvax8,ewmzr,9759243157894736,2419200.0,NaN
1,1,668d393e858e8126275433046bbd35c6tywop,aeqok,9759243157894736,2419200.0,0.0
2,2,aa5f360084278b35d746fa6af3a7a1a5ra3xe,wa00e,9759243157894736,2419200.0,NaN
3,3,3939ac3ef7d472a59a9c5f893dd3e39fh9ofi,jefix,9759243157894736,2419200.0,NaN
4,4,8393c48eaf4b8fa96886edc7cf27b372dsibi,jefix,9759243157894736,2419200.0,NaN


### time_normalized

In [ ]:
min_time = bids['time'].min()
bids['time_normalized'] = round((bids['time'] - min_time) * 19 / 1e9,2)

In [ ]:
pd.DataFrame(bids.time_normalized.unique())
bids.head()

### outcome

In [ ]:
bids['outcome'] = bids.merge(b_train, how='left', on='bidder_id').outcome

### other

In [ ]:
del bids['merchandise']

In [307]:
del bids['device']
del bids['country']
del bids['ip']
del bids['url']

In [309]:
bids.to_csv('data/bids_normalized_short.csv', sep=',', header=True, index=False)

# 2) b_train table

In [ ]:
del b_train['address']
del b_train['payment_account']

### mean_time_diff

In [ ]:
b_train['mean_time_diff'] = 0

In [ ]:
b_train.head()

In [ ]:
for bidder in b_train['bidder_id']:
    #d = bids[bids.bidder_id == bidder].sort_values('time_normalized')[['time_normalized']] - bids[bids.bidder_id == bidder].sort_values('time_normalized')[['time_normalized']].shift(1)
    #print(d.mean())
    #b_train.loc[b_train[b_train.bidder_id == bidder].index,'mean_time_diff'] = (bids[bids.bidder_id == bidder].sort_values('time_normalized')[['time_normalized']] - bids[bids.bidder_id == bidder].sort_values('time_normalized')[['time_normalized']].shift(1)).mean()
    
    m = (bids[bids.bidder_id == bidder].sort_values('time_normalized')[['time_normalized']] - 
         bids[bids.bidder_id == bidder].sort_values('time_normalized')[['time_normalized']].shift(1)).mean()[0]
    b_train.loc[b_train[b_train.bidder_id == bidder].index, 'mean_time_diff'] = m
    
#d['shifted'] = d.shift(1)
#d['difference'] = d['time'] - d['shifted']
#d
    #plt.plot(d)
#plt.show()
#d

In [ ]:
b_train.loc[:,'mean_time_diff'] = round(b_train.mean_time_diff,5)

### mean_time_diff2

In [ ]:
b_train['mean_time_diff2'] = 0

In [ ]:
b_train.head()

In [ ]:
i = 0
for bidder in b_train['bidder_id'][:1]:
    if i % 10 == 0:
        print(i)
        print('bidder:', bidder)
        print(bids[bids.bidder_id == bidder].sort_values(by='time_normalized')[['auction','time_normalized']].count()[0])
    i += 1
    time_diffs = []
    for auction in bids[bids.bidder_id == bidder].sort_values(by='time_normalized')[['auction','time_normalized']].values:
        print(auction)
        df = bids[bids.auction == auction[0]].sort_values(by='time_normalized')[['bidder_id', 'time_normalized']]
        df['diff'] = bids[bids.auction == auction[0]].sort_values(by='time_normalized')['time_normalized'] - bids[bids.auction == auction[0]].sort_values(by='time_normalized')['time_normalized'].shift(1)
        c = np.array(df[(df.bidder_id == bidder) & (df.time_normalized == auction[1])]['diff'])[0]
        time_diffs.append(c)
    b_train.loc[b_train[b_train.bidder_id == bidder].index, 'mean_time_diff2'] = np.array(time_diffs).mean()

In [355]:
bids_sorted = bids.sort_values(by=['auction', 'time'])
del bids_sorted['time']
del bids_sorted['outcome']
bids_sorted['diff'] = bids_sorted.time_normalized - bids_sorted.time_normalized.shift(1)

In [ ]:
i = 0
for auc in bids_sorted.auction.unique():
    print(i)
    i += 1
    bids_sorted.loc[bids_sorted[bids_sorted.auction == auc].index, 'diff2'] = bids_sorted[bids_sorted.auction == auc].time_normalized - bids_sorted[bids_sorted.auction == auc].time_normalized.shift(1)

In [376]:
bids_sorted.to_csv('data/bids_sorted.csv', sep=',', header=True, index=False)

In [373]:
i = 0
for auc in bids_sorted.auction.unique():
    if i % 200 == 0:
        print(i)
    i += 1
    bids_sorted.loc[bids_sorted[bids_sorted.auction == auc].index[0], 'diff'] = np.NaN
    #bids_sorted.loc[bids_sorted[bids_sorted.auction == auc].index, 'diff'] = bids_sorted[bids_sorted.auction == auc].time_normalized - bids_sorted[bids_sorted.auction == auc].time_normalized.shift(1)

0
200
400
600
800
1000
1200
1400
1600
1800
2000
2200
2400
2600
2800
3000
3200
3400
3600
3800
4000
4200
4400
4600
4800
5000
5200
5400
5600
5800
6000
6200
6400
6600
6800
7000
7200
7400
7600
7800
8000
8200
8400
8600
8800
9000
9200
9400
9600
9800
10000
10200
10400
10600
10800
11000
11200
11400
11600


KeyboardInterrupt: 

In [371]:
bids_sorted

,bid_id,bidder_id,auction,time_normalized,diff
5826422,5826422,92af1e40713e077ef87f5352fb56772fjnzm7,00270,1275528.0,NaN
5834284,5834284,a939f51234ad2c44eb9ccc84e754f41foiv7g,00270,1276446.0,918.0
5850448,5850448,92af1e40713e077ef87f5352fb56772fjnzm7,00270,1278300.0,1854.0
5884054,5884054,9655ccc7c0c193f1549475f02c54dce45kjw7,00270,1282037.0,3737.0
5898318,5898318,9655ccc7c0c193f1549475f02c54dce45kjw7,00270,1283595.0,1558.0
5910348,5910348,9655ccc7c0c193f1549475f02c54dce45kjw7,00270,1284896.0,1301.0
5911709,5911709,84c4b58a1491da3a30710bcdf61f87054xfq6,00270,1285043.0,147.0
5915604,5915604,92af1e40713e077ef87f5352fb56772fjnzm7,00270,1285445.0,402.0
5921974,5921974,9a81137ad31d7253759cdb9ae8e44227fw8x9,00270,1286085.0,640.0
5925427,5925427,9655ccc7c0c193f1549475f02c54dce45kjw7,00270,1286425.0,340.0


### total_bids_count

In [ ]:
b_train['total_bids_count'] = 0

In [211]:
b_train.head()

,bidder_id,outcome,mean_time_diff,total_bids_count
0,91a3c57b13234af24875c56fb7e2b2f4rb56a,0.0,10851.13,24
1,624f258b49e77713fc34034560f93fb3hu3jo,0.0,61438.00,3
2,1c5f4fc669099bfbfac515cd26997bd12ruaj,0.0,45201.00,4
3,4bee9aba2abda51bf43d639013d6efe12iycd,0.0,NaN,1
4,4ab12bc61c82ddd9c2d65e60555808acqgos1,0.0,1477.80,155


In [210]:
i = 0
for bidder in b_train['bidder_id']:
    if i % 100 == 0:
        print(i)
    i += 1
    c = bids[bids.bidder_id == bidder].count()[0]
    b_train.loc[b_train[b_train.bidder_id == bidder].index, 'total_bids_count'] = c

1100
1200
1300
1400
1500
1600
1700
1800
1900
2000


### total_auctions_count

In [213]:
b_train['total_auctions_count'] = 0

In [216]:
b_train.head()

,bidder_id,outcome,mean_time_diff,total_bids_count,total_auctions_count
0,91a3c57b13234af24875c56fb7e2b2f4rb56a,0.0,10851.13,24,18
1,624f258b49e77713fc34034560f93fb3hu3jo,0.0,61438.00,3,1
2,1c5f4fc669099bfbfac515cd26997bd12ruaj,0.0,45201.00,4,4
3,4bee9aba2abda51bf43d639013d6efe12iycd,0.0,NaN,1,1
4,4ab12bc61c82ddd9c2d65e60555808acqgos1,0.0,1477.80,155,23


In [215]:
i = 0
for bidder in b_train['bidder_id']:
    if i % 100 == 0:
        print(i)
    i += 1
    c = bids[bids.bidder_id == bidder]['auction'].nunique()
    b_train.loc[b_train[b_train.bidder_id == bidder].index, 'total_auctions_count'] = c

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000


In [217]:
b_train.to_csv('data/b_train.csv', sep=',', header=True, index=False)